In [36]:
import pandas as pd
import numpy as np
from scipy.stats import norm

In [37]:
data = {
    'Outlook': ['Sunny', 'Sunny', 'Overcast', 'Rain', 'Rain', 'Rain', 'Overcast', 'Overcast', 'Sunny', 'Rain'],
    'Temperature': ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Cool', 'Mild', 'Cool', 'Mild'],
    'Humidity': ['High', 'High', 'High', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'Normal'],
    'Wind': ['Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Weak', 'Weak'],
    'PlayTennis': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes']
}
df = pd.DataFrame(data)
X = df[['Outlook', 'Temperature', 'Humidity', 'Wind']]
y = df['PlayTennis']

In [38]:
X = df[['Outlook', 'Temperature', 'Humidity', 'Wind']]
y = df['PlayTennis']

In [39]:
class NaiveClassifier:
    def __init__(self):
        self.prior = {}
        self.conditional = {}
        self.feature_types = {}

    def fit(self, X: pd.DataFrame, y: pd.DataFrame):
        self.prior = y.value_counts(normalize=True).to_dict()
        

        for feature in X.columns:
            self.feature_types[feature] = 'continuous' if X[feature].dtype in ['int64', 'int16','int8','float64', 'float32'] else 'categorical' 
            self.conditional[feature] = {}
            
            if self.feature_types[feature] == 'categorical':
                unique_value_of_feat = X[feature].unique()
                for item in unique_value_of_feat:
                    self.conditional[feature][item] = {}
                    for class_label in y.unique():
                        self.conditional[feature][item][class_label] = X[y == class_label][feature].value_counts(normalize=True).to_dict()
            else:
                for class_label in y.unique():
                    class_data = X[y == class_label][feature]
                    self.conditional[feature][class_label] = {
                        'mean': class_data.mean(),
                        'std': max(class_data.std(), 1e-6)
                    }
        
    def predict(self, x):
        probabilities = {}
        for class_label in self.prior.keys():
            prob = self.prior[class_label]
            for feature, value in x.items():
                if self.feature_types[feature] == 'categorical':
                    prob *= self.conditional[feature][value][class_label].get(value, 00000.1)
                else:  # continuous feature
                    mean = self.conditional[feature][class_label]['mean']
                    std = self.conditional[feature][class_label]['std']
                    prob *= norm.pdf(value, mean, std)
                
            
            probabilities[class_label] = prob
        
        return probabilities

                

In [40]:
X['Outlook'].unique()

array(['Sunny', 'Overcast', 'Rain'], dtype=object)

In [41]:
X = df[['Outlook', 'Temperature', 'Humidity', 'Wind']]
y = df['PlayTennis']

In [42]:
clf = NaiveClassifier()
clf.fit(X, y)


In [43]:
X_test = {
    'Outlook': 'Sunny',
    'Temperature': 'Cool',
    'Humidity': 'High',
    'Wind': 'Strong'
}
clf.predict(X_test)

{'Yes': 0.002777777777777777, 'No': 0.018750000000000003}

# Multi-class

In [44]:
weather_class_features = {
    "Day": ["Weekday", "Weekday", "Weekday", "Holiday", "Saturday", "Weekday", "Holiday", "Sunday", 
            "Weekday", "Weekday", "Saturday", "Weekday", "Weekday", "Weekday", "Weekday", "Saturday", 
            "Weekday", "Holiday", "Weekday", "Weekday"],
    "Season": ["Spring", "Winter", "Winter", "Winter", "Summer", "Autumn", "Summer", "Summer", 
               "Winter", "Summer", "Spring", "Summer", "Winter", "Summer", "Winter", "Autumn", 
               "Autumn", "Spring", "Spring", "Spring"],
    "Fog": ["None", "None", "None", "High", "Normal", "Normal", "High", "Normal", 
            "High", "None", "High", "High", "Normal", "High", "Normal", "High", 
            "None", "Normal", "Normal", "Normal"],
    "Rain": ["None", "Slight", "None", "Slight", "None", "None", "Slight", "None", 
             "Heavy", "Slight", "Heavy", "Slight", "None", "None", "Heavy", "Slight", 
             "Heavy", "Slight", "None", "Heavy"],
    "Class": ["On Time", "On Time", "On Time", "Late", "On Time", "Very Late", "On Time", "On Time", 
              "Very Late", "On Time", "Cancelled", "On Time", "Late", "On Time", "Very Late", "On Time", 
              "On Time", "On Time", "On Time", "On Time"]
}
df_2 = pd.DataFrame(weather_class_features)

In [45]:
X = df_2[['Day', 'Season', 'Fog', 'Rain']]
y = df_2['Class']

In [46]:
clf_2 = NaiveClassifier()
clf_2.fit(X, y)


In [47]:
clf_2.prior

{'On Time': 0.7, 'Very Late': 0.15, 'Late': 0.1, 'Cancelled': 0.05}

In [48]:
X_test = {
    'Day': 'Weekday', 
    'Season': 'Winter',
    'Fog': 'High', 
    'Rain': 'Heavy'

}

In [49]:
clf_2.predict(X_test)

{'On Time': 0.0026239067055393583,
 'Very Late': 0.022222222222222216,
 'Late': 0.0025000000000000005,
 'Cancelled': 0.0005000000000000001}

# Iris

In [50]:
data = {
    "Length": [1.4, 1.0, 1.3, 1.9, 2.0, 1.8, 3.0, 3.8, 4.1, 3.9, 4.2, 3.4],
    "Class": [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
}
df_iris = pd.DataFrame(data)

In [51]:
X = df_iris[['Length']]
y = df_iris['Class']
X

,Length
0,1.4
1,1.0
2,1.3
3,1.9
4,2.0
5,1.8
6,3.0
7,3.8
8,4.1
9,3.9


In [52]:
clf_iris = NaiveClassifier()
clf_iris.fit(X, y)

In [53]:
clf_iris.conditional['Length'][1]

{'mean': np.float64(3.733333333333333), 'std': np.float64(0.4546060565661952)}

In [54]:
X_test = {
    "Length" : 3.4
}

In [55]:
clf_iris.predict(X_test)

{0: np.float64(9.68913769105268e-06), 1: np.float64(0.3353505510396091)}

In [56]:
data = {
    'Outlook': ['Sunny', 'Sunny', 'Overcast', 'Rain', 'Rain', 'Rain', 'Overcast', 'Overcast', 'Sunny', 'Rain'],
    'Temperature': ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Cool', 'Mild', 'Cool', 'Mild'],
    'Humidity': ['High', 'High', 'High', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'Normal'],
    'Wind': ['Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Weak', 'Weak'],
    'PlayTennis': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes']
}
df = pd.DataFrame(data)
X = df[['Outlook', 'Temperature', 'Humidity', 'Wind']]
y = df['PlayTennis']

In [57]:
clstmp = NaiveClassifier()
clstmp.fit(X, y)

In [58]:
X_test = {
    'Outlook': 'Sunny', 
    'Temperature': 'Cool',
    "Humidity": 'High', 
    'Wind': 'Strong', 
}
clstmp.predict(X_test)

{'Yes': 0.002777777777777777, 'No': 0.018750000000000003}

In [59]:
clstmp.conditional['Outlook']['Sunny']['No']

{'Sunny': 0.5, 'Rain': 0.25, 'Overcast': 0.25}

In [60]:
from sklearn.datasets import load_iris

In [61]:
data = {
    'Tuổi': ['Trẻ', 'Trẻ', 'Trung niên', 'Già', 'Già', 'Trung niên', 'Trẻ', 'Trung niên', 'Già', 'Trẻ'],
    'Thu nhập': ['Thấp', 'Trung bình', 'Thấp', 'Trung bình', 'Cao', 'Cao', 'Cao', 'Trung bình', 'Thấp', 'Trung bình'],
    'Mua bảo hiểm': ['Không', 'Có', 'Có', 'Có', 'Không', 'Có', 'Không', 'Có', 'Có', 'Không']
}
df = pd.DataFrame(data)
X = df[['Tuổi', 'Thu nhập']]
y = df['Mua bảo hiểm']

In [62]:
clstmp = NaiveClassifier()
clstmp.fit(X, y)

In [63]:
X_test = {
    'Tuổi': 'Trung niên',
    'Thu nhập': 'Cao'
}
clstmp.predict(X_test)

{'Có': 0.049999999999999996, 'Không': 0.020000000000000004}

In [66]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder

# Prepare the data
data = {
    'Cường độ ánh sáng (lumen)': [805, 780, 795, 820, 810, 785, 815, 800, 610, 590, 620, 580, 600, 595, 605, 585],
    'Tuổi thọ (giờ)': [51000, 49500, 52000, 50500, 48000, 53000, 49000, 50000, 31000, 28000, 33000, 29000, 32000, 27000, 34000, 31500],
    'Phân loại': ['Đạt', 'Đạt', 'Đạt', 'Đạt', 'Đạt', 'Đạt', 'Đạt', 'Đạt', 'Không đạt', 'Không đạt', 'Không đạt', 'Không đạt', 'Không đạt', 'Không đạt', 'Không đạt', 'Không đạt']
}

df = pd.DataFrame(data)

# Encode the labels
label_encoder = LabelEncoder()
df['Phân loại'] = label_encoder.fit_transform(df['Phân loại'])  # 0 for 'Đạt', 1 for 'Không đạt'

# Features and target
X = df[['Cường độ ánh sáng (lumen)', 'Tuổi thọ (giờ)']]
y = df['Phân loại']

# Train the model
model = GaussianNB()
model.fit(X, y)

# Predict the probability for new data
new_data = np.array([[750, 45000]])
probabilities = model.predict_proba(new_data)

# Get P(Đạt | Cường độ ánh sáng = 750, Tuổi thọ = 45000)
p_dat = probabilities[0][0]  # Probability of 'Đạt' (class 0)
print(f"P(Đạt | Cường độ ánh sáng = 750, Tuổi thọ = 45000) = {p_dat:.100f}")
p_dat

P(Đạt | Cường độ ánh sáng = 750, Tuổi thọ = 45000) = 0.0000000000000000000000000000000000231487160951319045273859278144324892855301787680190114554758964491


d:\BachKhoa\AIO\AIO-Exercise-And-Note\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


np.float64(2.3148716095131905e-35)